# Predicting NBA Player Career Duration

In [2]:
%load_ext lab_black
import pandas as pd
import numpy as np

Load data

In [3]:
player_careers = pd.read_csv("../data/NBA_all_seasons/Player Career Info.csv")
player_stats = pd.read_csv("../data/NBA_all_seasons/Player Per Game.csv")

Filter out the players whose careers were drafted after 2017 (too early to evaluate) 

In [4]:
player_careers_draft_before_2017 = player_careers[player_careers["first_seas"] <= 2017]

In [5]:
player_careers_draft_before_2017

,player_id,player,birth_year,hof,num_seasons,first_seas,last_seas
0,1,Al Brightman,NaN,False,1,1947,1947
1,2,Al Lujack,NaN,False,1,1947,1947
2,3,Al Negratti,NaN,False,1,1947,1947
3,4,Angelo Musi,NaN,False,3,1947,1949
4,5,Ariel Maughan,1923.0,False,5,1947,1951
...,...,...,...,...,...,...,...
4456,4457,Tyler Ulis,NaN,False,3,2017,2019
4457,4458,Wade Baldwin,NaN,False,3,2017,2019
4458,4459,Wayne Selden,NaN,False,4,2017,2022
4459,4460,Willy Hernangómez,NaN,False,6,2017,2022


Create target label for career longer than 5 years

In [6]:
player_careers_draft_before_2017[
    "long_career"
] = player_careers_draft_before_2017.apply(
    lambda row: 1 if row["num_seasons"] >= 5 else 0, axis=1
)

/var/folders/14/4hhh9__50mq6_c8ms3nqzrbr0000gn/T/ipykernel_83304/3562907748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_careers_draft_before_2017[


Right join the player stats to filter out class of 2017 after

In [7]:
players_data = pd.merge(
    player_careers_draft_before_2017.drop(["player", "birth_year"], axis=1),
    player_stats.drop(["birth_year"], axis=1),
    how="left",
    on="player_id",
).dropna()

Filter to all players 3rd year or earlier

In [8]:
players_rookie_contracts = players_data[players_data["experience"] <= 3]

In [11]:
players_rookie_contracts.sort_values("experience", ascending=False).groupby(
    "player_id"
).head(1)

,player_id,hof,num_seasons,first_seas,last_seas,long_career,seas_id,season,player,pos,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
14760,2585,True,17,1991,2007,1,13308,1993,Gary Payton,PG,...,0.770,1.2,2.3,3.4,4.9,2.2,0.3,1.8,3.0,13.5
24344,3783,False,12,2008,2019,1,22570,2010,Nick Young,SG,...,0.800,0.3,1.1,1.4,0.6,0.4,0.1,0.8,2.0,8.6
15510,2684,False,15,1992,2006,1,14022,1994,Stacey Augmon,SG,...,0.764,2.2,2.6,4.8,2.3,1.8,0.5,1.8,2.2,14.8
24376,3786,False,10,2008,2017,1,22619,2010,Rodney Stuckey,PG,...,0.833,0.9,2.9,3.8,4.8,1.4,0.2,2.2,2.8,16.6
24366,3785,False,11,2008,2018,1,22602,2010,Ramon Sessions,PG,...,0.717,0.7,1.9,2.6,3.1,0.7,0.1,1.7,1.7,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15694,2710,False,1,1993,1993,0,13275,1993,Dexter Cambridge,SF,...,0.687,1.7,1.5,3.2,1.1,0.5,0.1,1.2,2.4,7.0
24111,3755,False,1,2008,2008,0,21193,2008,Guillermo Díaz,SG,...,0.600,0.3,0.0,0.3,0.2,0.0,0.0,0.0,0.0,0.8
24070,3747,False,1,2008,2008,0,21105,2008,D.J. Strawberry,SG,...,0.474,0.3,0.6,0.8,0.9,0.4,0.2,0.3,1.1,2.2
15753,2717,False,1,1993,1993,0,13325,1993,Gundars Vētra,SF,...,0.667,0.3,0.3,0.6,0.5,0.2,0.0,0.2,0.9,3.5
